<img src="https://www.dbs.ie/images/default-source/logos/dbs-logo-2019-small.png" align = left/>

#  Open University Learning Analytics - Student Dropout

Capstone Project

Claire Connaughton (10266499)

# Import Relevant Libraries

In [ ]:
import os
import pickle
import pydotplus
import numpy as np
import pandas as pd
from functools import reduce
from plotnine import *
import plotnine
plotnine.options.figure_size = (5.2,3.2)
import seaborn as sns
sns.set()
sns.set_style("white")
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from scipy import interp
from scipy.stats import skew, norm, probplot, boxcox, f_oneway
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import make_column_transformer
from IPython.display import Image  
from sklearn.tree import export_graphviz
from six import StringIO
from collections import Counter
from pandas_profiling import ProfileReport
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split 
from sklearn.utils import resample 
from imblearn.over_sampling import SMOTE 
from sklearn.metrics import confusion_matrix 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score, mean_absolute_error
from sklearn import metrics, tree
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

# Import dataset

In [ ]:
# Load the cleaned OULA dataset

try:
    data = pd.read_csv('oulad_cleaned.csv')
    print("The 'Cleaned Dropout OULA' dataset has {} samples with {} features each.".format(*data.shape))
except:
    print("Dataset could not be loaded. Is the dataset missing?")

In [ ]:
data.info()

In [ ]:
# Remove the columns which are not required for dropout analysis

data.drop(columns=['student_failed'], inplace=True)


In [ ]:
# Inpsect the number of students who dropped out

g = sns.countplot(x ='dropout', 
              data = data,
              color='grey',
              order = data.dropout.value_counts().index);

patch_h = []    
for patch in g.patches:
    reading = patch.get_height()
    patch_h.append(reading)

idx_tallest = np.argmax(patch_h)   

g.patches[idx_tallest].set_facecolor('#a834a8') 
g.set_xticklabels(['Completed Course','Dropped Out'])
g.set_title('Student Dropouts', fontsize = 18)


In [ ]:
# Clean the code_module column

# There are 7 course modules. 4 are from STEM and 3 from Social Sciences
# Social Sciences :- AAA, BBB, GGG
# STEM :- CCC, DDD, EEE, FFF
# Create two categories instead of 7


data['code_module'] = ['Social_Science' if data['code_module'].iloc[i] in ['AAA', 'BBB', 'GGG']
                                                        else 'STEM' for i in range(len(data))]

In [ ]:
#Inpsect the code module category

g = sns.countplot(x ='code_module', 
              data = data,
              color='grey',
              order = data.code_module.value_counts().index);

patch_h = []    
for patch in g.patches:
    reading = patch.get_height()
    patch_h.append(reading)

idx_tallest = np.argmax(patch_h)   

g.patches[idx_tallest].set_facecolor('#a834a8')  
g.set_title('Code Module', fontsize = 18)


# Univariate Analysis

In [ ]:
data.describe().transpose()

In [ ]:
# Make a dataframe with just the categorical variables

categoricals= data.select_dtypes(exclude=np.number)

In [ ]:
# Set the plot number for the first subplot function
plot_number = 1

# Set sizes for all plots
plt.figure(figsize=(15, 15)) # create a figure object
plt.subplots_adjust(hspace = 0.5) # set the size of subplots

for col in categoricals:
    
    # Call countplot on each column
    plt.subplot(6, 2, plot_number)
    sns.countplot(
        y=col,
        data=categoricals,
        order=categoricals[col].value_counts().index
    )
    plt.title(f'{col.capitalize()} Countplot')
    plt.xlabel('')
    plt.ylabel('')

    plot_number = plot_number + 1 # set a new plot number for the next subplot function
    
    # Add relative frequency labels:
    n_points = categoricals.shape[0]
    col_counts = categoricals[col].value_counts()
    locs, labels = plt.yticks()   # get the current tick locations and labels

    # loop through each pair of locations and labels
    for loc, label in zip(locs, labels):

        # get the text property for the label to get the correct count
        count = col_counts[label.get_text()]
        pct_string = '{:0.1f}%'.format(100*count/n_points)

        # print the annotation at the top of the bar
        plt.text(x=count, y=loc, s=pct_string, ha='left', va='center', color='k')
    
plt.tight_layout()

# Bivariate Analysis

In [ ]:
sns.pairplot(data)

The results are vague. There doesn't seem to be any strong relationships between any variables and the target except from the number of total clicks.

In [ ]:
corr=data.apply(lambda x:pd.factorize(x)[0]).corr(method='pearson')
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(corr,annot=True, mask=mask, cmap="YlGnBu")
plt.title("Multi Variate Correlation Plot", fontsize=20);

The stongest positive correlations were between dropout unreg_month (0.8). 

There were also strong relationships total_late_submission and late_rate (0.43), AVG_click and procrastination (0.46). 

The highest negative correlation was observed between gender and code_module (-0.56).


# Bivariate Analysis for Student Dropouts

***************************

# Demographic Details of Students who Dropout

In [ ]:
# Inspect the number of dropouts in each module

g= sns.catplot(y= 'dropout', col='code_module', col_wrap=4,
                data=data[data.code_module.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.fig.subplots_adjust(top=0.9) 
g.set_yticklabels(['Completed Course','Dropout'])

More students dropped out of STEM courses

In [ ]:
# Check if there is a gender difference in dropouts

g= sns.catplot(y= 'dropout', col='gender', col_wrap=4,
                data=data[data.gender.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.fig.subplots_adjust(top=0.9) 
g.set_yticklabels(['Completed Course','Dropout'])

Slightly more males dropped out.

In [ ]:
# Check if there is a regional difference in dropouts

g= sns.catplot(y= 'dropout', col='region', col_wrap=4,
                data=data[data.region.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.fig.subplots_adjust(top=0.9) 
g.set_yticklabels(['Completed Course','Dropout'])

South UK had fewer dropouts than any other region.

In [ ]:
# Check if there is difference in highest education level among dropouts

g= sns.catplot(y= 'dropout', col='highest_education', col_wrap=4,
                data=data[data.highest_education.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_yticklabels(['Completed Course','Dropout'])

Students with a higher education qualification were the least likely to dropout and students with the lowest educational attainment are the most likely to dropout.

In [ ]:
# Check if there is difference in socio-economic status among dropouts

g= sns.catplot(y= 'dropout', col='imd_band', col_wrap=4,
                data=data[data.imd_band.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_yticklabels(['Completed Course','Dropout'])

Socioeconomically disadvantaged students were more likely to dropout. The most economically priviledged students had the lowest dropout rates.

In [ ]:
# Check if there is an age difference among dropouts
g= sns.catplot(y= 'dropout', col='age_band', col_wrap=4,
                data=data[data.age_band.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_yticklabels(['Completed Course','Dropout'])


Under 35s are the most likely to dropout of their course.

In [ ]:
# Check if there is a difference among student ability groups and dropouts

g= sns.catplot(y= 'dropout', col='disability', col_wrap=4,
                data=data[data.disability.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_yticklabels(['Completed Course','Dropout'])

Students with a disability are more likely to dropout. 

# The demographic profile of students who are more likely to dropout is as follows:

1) Under 35s


2) Male


3) Not being from Northern UK 


4) Studying a STEM course


5) From a lower socio-economic class


6) Holding lower than A-Level education


# Behavioural Details of Students who Dropout

In [ ]:
# Investigate the relationship between dropping out and the number of previous attempts

g= sns.catplot(y= 'dropout', col='num_of_prev_attempts', col_wrap=4,
                data=data[data.num_of_prev_attempts.notnull()],
                kind="count", height=3.5, aspect=.8, 
                palette= "tab20")
g.set_yticklabels(['Completed Course','Dropout'])

Students who dropped out made more attempts to complete the module. 

In [ ]:
# However, very few students made another attempt

g = sns.countplot(x ='num_of_prev_attempts', 
              data = data,
              color='grey',
              order = data.num_of_prev_attempts.value_counts().index);

patch_h = []    
for patch in g.patches:
    reading = patch.get_height()
    patch_h.append(reading)

idx_tallest = np.argmax(patch_h)   

g.patches[idx_tallest].set_facecolor('c')  
g.set_title('Number of Previous Attempts', fontsize = 18)

In [ ]:
# Investigate the relationship between dropping out and the number of studied credits

plt.figure(figsize=(8,4))
sns.barplot(y='studied_credits',x='dropout',data=data,palette='Blues_d');
plt.title("Number of Studied Credits by Dropouts", fontsize=18);
plt.xticks(np.arange(2),('Completed Course','Dropped Out'));


The students who dropped out studied more credits. 

In [ ]:
# Investigate the relationship between dropping out and the Average number of clicks

plt.figure(figsize=(8,4))
sns.barplot(y='AVG_click',x='dropout',data=data,palette='Blues_d');
plt.title("Impact AVG Clicks on Dropping Out", fontsize=18);
plt.xticks(np.arange(2),('Completed Course','Dropped Out'));


Students who dropped out had a lower number of average clicks on the Virtual Learning Environment.

In [ ]:
# Investigate the relationship between dropping out and the total clicks in the VLE

plt.figure(figsize=(8,4))
sns.barplot(y='total_click',x='dropout',data=data,palette='Blues_d');
plt.title("Impact Total Clicks by Dropouts", fontsize=18);
plt.xticks(np.arange(2),('Completed Course','Dropped Out'));

The students who dropped out had considerably fewer total clicks than the students who completed the course.

In [ ]:
# Investigate the relationship between dropping out and late submission of assignments

plt.figure(figsize=(8,4))
sns.barplot(y='late_rate',x='dropout',data=data,palette='Blues_d');
plt.title("Impact Late Submission on Dropping Out", fontsize=18);
plt.xticks(np.arange(2),('Completed Course','Dropped Out'));

Students who dropped out were more likely to submit their assignment late.

In [ ]:
# Investigate the relationship between dropping out and weighted score

plt.figure(figsize=(8,4))
sns.barplot(y='weighted_score',x='dropout',data=data,palette='Blues_d');
plt.title("Impact Weighted Score on Dropping Out", fontsize=18);
plt.xticks(np.arange(2),('Completed Course','Dropped Out'));

Students who had lower weighted scores had higher dropout rates.

In [ ]:
# Investigate the relationship between dropping out and the date unregistered

# Visulaise the breakdown in module lenght per module type

pd.crosstab(data.unreg_month, data.dropout).plot.barh(stacked = True);

Most dropouts occured in the first term.

# The behaviour profile of students who are more likely to dropout is as follows:

1) Studied more credits so their workload was higher

2) Less inclined to interact with the VLE


3) More likely to submit assignments late


4) Tended to have lower attainment scores

5) Were more likely to dropout in the first term



**********************************************

# Multivariate Analysis for Student Dropouts

In [ ]:
# Use NumPy's corrcoef and seaborn's heatmap functions to plot the correlation matrix array as a heat map.

corr = data.corr()
top_corr_cols = corr.dropout.sort_values(ascending=False).keys() 
top_corr = corr.loc[top_corr_cols, top_corr_cols]
dropSelf = np.zeros_like(top_corr)
dropSelf[np.triu_indices_from(dropSelf)] = True
plt.figure(figsize=(18, 10))
sns.heatmap(top_corr, cmap=sns.diverging_palette(220, 10, as_cmap=True), annot=True, fmt=".2f", mask=dropSelf)
sns.set(font_scale=1.5)

sns.set(font_scale=1.0)
cols = data.columns
g = sns.pairplot(data = data.loc[:, cols], hue='dropout')
fig = g.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Pairwise Plots by Dropout', fontsize=24)

plt.show()

del corr, dropSelf, top_corr, g, fig, t

The multivariate analysis provides further confirmation that the students who dropped out were academically struggling and had lower engagement with the VLE.

The multivariate analysis also indicates that some columns are providing the same information, such as AVG_click and total_click, total_late_submission and late_rate.

These columns should be dropped.

In [ ]:
data.drop(columns=['AVG_click', 'total_assessments', 'total_late_submission'], axis = 1, inplace = True)

# MODEL PREPARATION

Checking the Variance Inflation Factor for Each Variable

In [ ]:
# Find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('dropout ~ code_module+code_presentation+reg_month+unreg_month+gender+region+highest_education+imd_band+age_band+num_of_prev_attempts+disability+studied_credits+total_click+weighted_score+late_rate+fail_rate+procrastination', data=data, return_type='dataframe')

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif


In [ ]:
# Using 5 as the minimum vif values i.e any independent variable 5 and above will have to be dropped
# From the results all independent variable are below 5
# Drop reg_month column 

data.drop('reg_month', axis = 1, inplace = True)

In [ ]:
# Drop the ID column because it's unnecessary for model learning predictions

data.drop('id_student', axis = 1, inplace = True)

In [ ]:
#Declaring independent variable i.e X
#Declaring Target variable i.e y

x = data.drop(['dropout'], axis = 1)
y = data['dropout']


In [ ]:
# Set encoding and scaling instructions
column_transform = make_column_transformer(
    (OneHotEncoder(), ['code_module', 'code_presentation', 'gender', 'region', 'age_band', 'disability']),
    (OrdinalEncoder(), ['highest_education', 'imd_band']),
    (RobustScaler(), ['num_of_prev_attempts', 'studied_credits', 'total_click', 'late_rate'])
)

# Apply column transformer to features
X_encoded = column_transform.fit_transform(x)

In [ ]:
#Splitting data into train and test dataset

x_train, x_test, y_train, y_test = train_test_split(X_encoded, y , test_size = 0.2, random_state  = 365)

In [ ]:
# Set up cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=40) 

In [ ]:
# Set up SMOTE to treat the imbalanced target variable 
smote = SMOTE() 

In [ ]:
# Apply SMOTE
x_sm, y_sm =smote.fit_resample(x_train, y_train) 

print(x_train.shape, y_train.shape) 
print(x_sm.shape, y_sm.shape) 
sns.countplot(y_sm) 

# MODELLING

**********************************

# Logistic Regression

In [ ]:
# Define logistic regression model function
def logistic_regression(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.20, random_state=40, stratify = y) 
    logreg_model = LogisticRegression() 
    logreg_model.fit(x_train, y_train) 

    pred_train = logreg_model.predict(x_train) 
    pred_test = logreg_model.predict(x_test) 
    cm_train = confusion_matrix(y_train, pred_train) 
    cm_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  logreg_model.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call logistic regression model

logistic_regression(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# Decision Tree

In [ ]:
# Set up decision tree function 

def decision_tree(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.20, random_state=40, stratify = y) 
    dt = DecisionTreeClassifier() 
    dt.fit(x_train, y_train) 

    pred_train = dt.predict(x_train) 
    pred_test = dt.predict(x_test) 
    confusion_matrix_train = confusion_matrix(y_train, pred_train) 
    confusion_matrix_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  dt.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call decision tree model
decision_tree(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# Random Forest

In [ ]:
# Set up Random Forest function

def random_forest(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.20, random_state=40, stratify = y) 
    rf = RandomForestClassifier() 
    rf.fit(x_train, y_train) 

    pred_train = rf.predict(x_train) 
    pred_test = rf.predict(x_test) 
    confusion_matrix_train = confusion_matrix(y_train, pred_train) 
    confusion_matrix_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  rf.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call random forest function
random_forest(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# Support Vector Machine

In [ ]:
# Set up Support Vector Machine function

def support_vector_machine(x, y): 
    x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.25, random_state=40, stratify = y) 
    svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_train, y_train) 
    svm_predictions = svm_model_linear.predict(x_test)

    pred_train = svm_model_linear.predict(x_train) 
    pred_test = svm_model_linear.predict(x_test) 
    confusion_matrix_train = confusion_matrix(y_train, pred_train) 
    confusion_matrix_test = confusion_matrix(y_test, pred_test)
    score = round(accuracy_score(y_test, pred_test), 3)
    cm1 = confusion_matrix(y_test, pred_test)
    sns.heatmap(cm1, annot=True, fmt=".0f")
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.title('Accuracy Score: {0}'.format(score), size = 15)
    plt.show()
    print("Accuracy of Test Model : ",  svm_model_linear.score(x_test, y_test)) 
    print("Mean Absolute Error : ", mean_absolute_error(pred_test, y_test))
    print("Train Data Set") 
    print(metrics.classification_report(y_train,pred_train) ) 
    print("Test Data Set ") 
    print(metrics.classification_report(y_test,pred_test) ) 
    return  None 

In [ ]:
# Call support vector machine function

support_vector_machine(x_sm, y_sm) 
warnings.filterwarnings('ignore')

# XGBoost

In [ ]:
# Train and test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25)

In [ ]:
# Set up xgBoost matrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

In [ ]:
# Set parameters
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 50  # the number of training iterations

In [ ]:
# training and testing - numpy matrices
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)
# extracting most confident predictions
best_preds = np.asarray([np.argmax(line) for line in preds])
print ("Numpy array precision:", precision_score(y_test, best_preds, average='macro'))

In [ ]:
# Print model metrics 
print(accuracy_score(y_test, best_preds))
print(recall_score(y_test, best_preds, average='macro'))
print(f1_score(y_test, best_preds, average='macro'))
print(precision_score(y_test, best_preds, average='macro'))

In [ ]:
# Set up confusion matrix
score = round(accuracy_score(y_test, best_preds), 3)
cm1 = confusion_matrix(y_test, best_preds)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()
print(metrics.classification_report(y_test,best_preds) )

# Determine Feature Importance

In [ ]:
# Extract feature importance
model = ExtraTreesRegressor()
model.fit(X_encoded, y)

In [ ]:
print(model.feature_importances_)

In [ ]:
# Retrieve column names
X_encoded= pd.DataFrame(X_encoded) 
x_names= X_encoded.iloc[:,:16]
x_names=pd.DataFrame(x_names)
model = ExtraTreesRegressor()
model.fit(x_names,y)

In [ ]:
x_names.info()

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(16).plot(kind='barh')
plt.show()

The module type, the year of presentation, the unreg_month, gender and fail rate are the five most important features. 

# Comparison of Model Performance 

In [ ]:
# Plotting Bar Chart for Accuracy of different classifiers
sns.set_style("white")
plt.figure(figsize=(12, 3))
model_accuracies = [0.81, 0.76, 0.73, 0.72, 0.70, 0.69]
model_names = ['RandomForest','XGBoost','Logistic Regression', 'Decision Tree', 'Support Vector Machine', 'ANN - MLP']
g= sns.barplot(x=model_names, y=model_accuracies, color='grey');

patch_h = []    
for patch in g.patches:
    reading = patch.get_height()
    patch_h.append(reading)

idx_tallest = np.argmax(patch_h)   

g.patches[idx_tallest].set_facecolor('seagreen')  
g.set_title('Model Accuracy for Dropout', fontsize = 18)


# Attempt to Improve the Model Using PCA

In [ ]:
# Set up PCA
pca = PCA()
X_train = pca.fit_transform(X_train)
pca.explained_variance_ratio_

In [ ]:
# Plot the explained variance

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

Approximately 15 principal components explain 95% of the variance in the data. 

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with the first 15 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
dt = DecisionTreeClassifier() 
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print('Decision Tree accuracy score with the first 15 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

PCA reduces the accuracy of the models so it will be abandoned. 

# END